# Chapter 7.함수 데커레이터와 클로저

- 함수 데커레이터는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있음
- 데커레이터를 잘 사용하려면 클로저를 먼저 알아야 한다.
- nonlocal 키워드 : 데커레이터를 구현할 때 필요함.

### nonlocal vs global
- Global Variable : 전역변수. 파이썬 전체 영역에서 사용할 수 있는 변수

In [5]:
a = 1

def Fn():
    print(a)
    
Fn() #Fn 함수를 실행. 1출력
print(a) # a 전역변수 출력. 1출력
del a

1
1


- Local Variable : 지역변수. 지역범위에서만 영향을 주고 받을 수 있는 변수

In [7]:
def Fn():
    a = 1 #함수 Fn의 지역 변수 a
    print(a)
    
Fn() #Fn 함수를 실행. 1출력
print(a) #print할 a가 없다

1


NameError: name 'a' is not defined

- 전역 변수를 변경?

In [8]:
a = 1 #전역변수

def Fn():
    a = 2 #함수 Fn의 지역 변수. 함수 안에서 선언한 a는 함수를 벗어난 어떤 코드에도 영향을 주지 않는다.
    print(a)
    
Fn() #Fn 함수를 실행
print(a) # 전역변수 출력

2
1


- 전역 변수를 특정 함수에 가져오려면?

In [9]:
a = 1 #전역변수

def Fn():
    global a # 전역 변수 a를 사용하겠다는 선언
    a = 2 # 전역변수의 값이 1에서 2로 변경
    print(a)
    
Fn() #Fn 함수를 실행
print(a) # 전역변수 출력 (함수 Fn안에서 값이 변경)

2
2


- 전역 변수가 없는데 global로 전역 변수를 만드는 경우

In [11]:
def Fn():
    global a # 전역 변수 a를 사용하겠다는 선언
    a = 1 # 전역변수의 값이 1
    print(a)
    
Fn() #Fn 함수를 실행
print(a) # 전역변수 출력. a를 가리키는 전역변수는 없지만 함수 Fn에서 global로 a를 선언했기 때문에 전역변수가 됨,

1
1


In [13]:
del a

In [14]:
def Fn():
    a = 1 # 전역변수의 값이 1
    print(a)
    
Fn() #Fn 함수를 실행
print(a) # 전역변수가 없기 때문에 에러

1


NameError: name 'a' is not defined

- nonlocal 지역변수

In [15]:
def Fn():
    a = 1 # Fn 함수 안의 지역변수
    def Fn2():
        nonlocal a #지역변수를 불러오기 위해 nonlocal을 사용
        a = 2
    Fn2() #험수 Fn2 실행
    print(a) #2 출력
    
Fn() #함수 Fn 실행

2


In [16]:
def Fn():
    a = 1 # Fn 함수 안의 지역변수
    def Fn2():
        a = 2 #Fn2 안에서만 사용되는 변수 a
    Fn2() #험수 Fn2 실행
    print(a) #2 출력
    
Fn() #함수 Fn 실행

1


- 클로저는 콜백을 이용한 효율적인 비동기 프로그래밍과 함수형 스타일로 코딩하는데 필수적

## 7.1 데커레이터 기본 지식 

- 데커레이터는 다른 함수를 인수로 받는 callable(데커레이트된 함수)이다.
- 데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 callable 객체로 대체한다.

In [ ]:
@decorate
def target():
    print('running target()')

위 코드는 아래와 같이 작동한다

In [ ]:
def target():
    print('running target()')
    
target = decorate(target)

두 코드를 실행한 후 target함수는 꼭 원래의 target() 함수를 가르키는 것이 아니라, decorate(target)이 반환한 함수를 가르키게 된다

###### 예제 7-1

In [20]:
def deco(func):
    def inner():
        print('running inner()')
    return inner #deco()가 inner() 함수 객체를 반환한다.

@deco
def target(): #target()을 deco로 데커레이트 했다
    print('running target()')

In [21]:
target() #데커레이트된 target()을 호출하면 실제로는 inner()를 실행한다.

running inner()


In [23]:
target #target이 inner()를 가리키고 있다.

<function __main__.deco.<locals>.inner()>

- 데커레이터는 다른 함수를 인수로 전달해서 호출하는 일반적인 callable과 동일.
- 그렇지만 런타임에 프로그램 행위를 변경하는 메타프로그래밍을 할 때 데커레이터가 상당히 편리

## 7.2 파이썬이 데커레이터를 실행하는 시점 

- 데커레이터의 핵심 특징은 데커레이트된 함수가 정의된 직후에 실행되는 것.
- 파이썬이 모듈을 로딩하는 시점, 즉 임포트 타임에 실행된다.

###### 예제 7-2

In [28]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register #f1과 f2는 register로 데커레이터 되어 있음.
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main(): #registry를 먼저 출력하고 f1, f2, f3를 차례로 실행한다.
    print('running main()')
    print('registry -> ', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    print('----- main()함수 실행 시점 -----')
    main()

running register(<function f1 at 0x0000026A1893E9D8>)
running register(<function f2 at 0x0000026A1885A730>)
----- main()함수 실행 시점 -----
running main()
registry ->  [<function f1 at 0x0000026A1893E9D8>, <function f2 at 0x0000026A1885A730>]
running f1()
running f2()
running f3()


- 결과 해석  
running register(<function f1 at 0x0000026A1893EC80>) : f1에 데커레이터 씌울 때 출력  
running register(<function f2 at 0x0000026A1885A6A8>) : f2에 데커레이터 씌울 때 출력  
running main() : main()함수 실행했을 때 출력  
registry ->  [<function f1 at 0x0000026A1893EC80>, <function f2 at 0x0000026A1885A6A8>] : 데커레이터가 씌워질 때 추가됨  
running f1()  
running f2()  
running f3()  

In [31]:
import registration

running register(<function f1 at 0x0000026A1893EF28>)
running register(<function f2 at 0x0000026A18955620>)


- py파일로 만들어서 임포트하면, 임포트 할 때 첫 두줄이 출력된다
- 이 예제는 임포트 타임(import time)과 런타임(run time)의 차이를 명확히 보여준다

##### 데커레이터의 흔한 사용 방식과 예제 7-2와의 차이
- 일반적으로 데커레이터를 정의하는 모듈과 데커레이터를 적용하는 모듈은 분리해서 구현하는데, 여기서는 함께 정의 되어있음
- 실제 코드에서 대부분의 데커레이터는 내부 함수를 정의해서 반환하지만, 여기서는 인수로 전달된 함수와 동일한 함수를 반환

## 7.3 데커레이터로 개선한 전략 패턴

- 예제 6-6에서 가장 큰 문제는 best_prom()함수에 의해 사용되는 promos 리스트에 함수명을 반복해서 사용하는 것.
- 새로운 함수를 정의할 때마다 promos리스트에 추가를 해줘야하는 번거로움도 존재
- 이를 데커레이터를 통해 해결 가능

###### 예제 6-6

In [ ]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo] ##여기에 매번 수기로 추가해줘야 한다.

def best_promo(order):
    """최대로 할인 받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

###### 예제 7-3

In [30]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 100 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10%할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantitiy >= 20:
            dicount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.7
    return 0

def best_promo(order):
    """최대로 할인 받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)
    

##### 예제 7-3의 장점  
- 프로모션 전략 함수명이 특별한 형태(_promo로 끝나는 등)일 필요가 없다.
- @promotino 데커레이터는 함수의 목적을 알려주고, 어떤 프로모션을 배제할 경우, 데커레이터만 주석처리하면 된다.

## 7.4 변수 범위 규칙

###### 예제 7-4

In [32]:
def f1(a):
    print(a)
    print(b)

In [33]:
f1(3)

3


NameError: name 'b' is not defined

In [34]:
b = 6

In [35]:
f1(3)

3
6


In [36]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [37]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [38]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [39]:
f3(3)

3
6


In [40]:
b

9

In [41]:
f3(3)

3
9


## 7.5 클로저

In [45]:
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total  = sum(self.series)
        return total/len(self.series)

In [47]:
avg = Averager()
avg(10)

10.0

In [48]:
avg(11)

10.5

In [49]:
avg(12)

11.0

In [50]:
def make_averager():
    series = [] #자유변수. 지역 범위에 바인딩되어 있지 않은 변수를 의미.
    
    def averager(new_value):
        series.append(new_value)
        total  = sum(series)
        return total/len(series)
    
    return averager

In [51]:
avg = make_averager()
avg(10)

10.0

In [52]:
avg(11)

10.5

In [53]:
avg(12)

11.0

In [54]:
avg.__code__.co_varnames

('new_value', 'total')

In [55]:
avg.__code__.co_freevars

('series',)

In [56]:
avg.__closure__

(<cell at 0x0000026A1883F978: list object at 0x0000026A18948888>,)

In [58]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## 7.6 nonlocal 선언

###### 예제 7-13

In [59]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [60]:
avg = make_averager()

In [61]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [62]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [63]:
avg = make_averager()

In [65]:
avg(10)

10.0

## 7.7 간단한 데커레이터 구현하기

###### 예제 7-15

In [69]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [70]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n <2 else n * factorial(n-1)

In [71]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.13664865s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000000s] factorial(1) -> 1
[0.00000000s] factorial(2) -> 2
[0.00000000s] factorial(3) -> 6
[0.00000000s] factorial(4) -> 24
[0.00000000s] factorial(5) -> 120
[0.00000000s] factorial(6) -> 720
6! =  720


## 7.8 표준 라이브러리에서 제공하는 데커레이터

#### 7.8.1 functools.lru_cache()를 이용한 모메이제이션

###### 7-18 fibo_demo.py

In [73]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00099707s] fibonacci(4) -> 3
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00099730s] fibonacci(4) -> 3
[0.00099730s] fibonacci(5) -> 5
[0.00199437s] fibonacci(6) -> 8
8


In [74]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00099730s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00099730s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00099730s] fibonacci(6) -> 8
8


#### 7.8.2 단일 디스패치를 이용한 범용 함수

In [75]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [76]:
htmlize({1,2,3})

'<pre>{1, 2, 3}</pre>'

In [77]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [78]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch  # <1>
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)  # <2>
def _(text):            # <3>
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)  # <4>
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)  # <5>
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

## 7.9 누적된 데커레이터

- 하나의 함수 f()에 두 데커레이터 @d1과 @d2를 차례로 적용하면, 결과는 f = d1(d2(f))와 동일

## 7.10 매개변수화된 데커레이터

In [79]:
registry = []  

def register(func):  
    print('running register(%s)' % func) 
    registry.append(func) 
    return func 

@register 
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3(): 
    print('running f3()')

def main(): 
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

main()

running register(<function f1 at 0x0000026A18CA5AE8>)
running register(<function f2 at 0x0000026A189551E0>)
running main()
registry -> [<function f1 at 0x0000026A18CA5AE8>, <function f2 at 0x0000026A189551E0>]
running f1()
running f2()
running f3()


In [80]:
registry = set() 

def register(active=True): 
    def decorate(func):  
        print('running register(active=%s)->decorate(%s)' % (active, func))
        if active:  
            registry.add(func)
        else:
            registry.discard(func)  

        return func  
    return decorate  

@register(active=False) 
def f1():
    print('running f1()')

@register()  
def f2():
    print('running f2()')

def f3():
    print('running f3()')


running register(active=False)->decorate(<function f1 at 0x0000026A18B34C80>)
running register(active=True)->decorate(<function f2 at 0x0000026A18CA5268>)
